In [1]:
import sys
import requests
import matplotlib.pyplot as plt
import pandas as pd
import datetime
sys.path.append('project')

In [2]:
import driver

In [3]:
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from IPython.display import display, clear_output

In [4]:
# tweet

tweet = widgets.Text(placeholder='Your tweet here')

tweet.layout.margin = '0 auto 10px'

In [5]:
# location

location = widgets.Text(placeholder='Your location here')

location.layout.margin = '0 auto'

In [6]:
button_send = widgets.Button(
                description='Classify',
                tooltip='Classify',
                button_style = 'success',
                style={'description_width': 'initial'}
            )

button_send.layout.width = '130px'
button_send.layout.height = '50px'
button_send.layout.margin = '30px auto'

output = widgets.Output()

def on_button_clicked(event):
    with output:
        clear_output()
        print("Your Tweet: ")
        print(f"{tweet.value}")
        print("Your Location: ")
        print(f"{location.value}")
        coords = driver.bc_get_coords(location.value)
        lat = coords[0]
        long = coords[1]
        c = lat + "," + long
        url = 'https://api.tomorrow.io/v4/timelines?'
        key = 'bxz3fQWN6JZ0AeMHoDjt8KwMZkr9e2QD'
        querystring = {
        "location":c,
        "fields":["temperature", "cloudCover"],
        "units":"imperial",
        "timesteps":"1d",
        "apikey":key}
        response = requests.request("GET", url, params=querystring)
        results = response.json()['data']['timelines'][0]['intervals']
        df = pd.DataFrame()
        for daily_result in results:
            date = daily_result['startTime'][0:10]
            temp = round(daily_result['values']['temperature'])
            cloud = round(daily_result['values']['cloudCover'])
            df = df.append({'dates':date, 'temp':temp, 'cloud':cloud}, ignore_index=True)
        week = df.head(5)
        plt.plot(week['dates'],week['temp'],week['cloud'])
        plt.xlabel("Date")
        plt.legend(['Temperature(F)', 'Cloud Cover'])
        plt.title("Weekly Temperature")
        plt.show()
        print(driver.classify(tweet.value, location.value))    

button_send.on_click(on_button_clicked)

vbox_result = widgets.VBox([output])

In [7]:
#text
header_text = widgets.HTML(value="<h1 style='color: white;'><center>Tweet Classifier</center></h1>")
tweet_label = widgets.HTML(value="<h2 style='color: white;'><center>Please Enter your Tweet</center></h2>")
location_label = widgets.HTML(value="<h2 style='color: white;'><center>Please Enter your Location</center></h2>")

#images
logo_image = widgets.HTML(value='<center><img src="Logo.png" width="300px"/></center>')
info_image = widgets.HTML(value='<center><img src="info.png" width="50px"/></center>')
location_image = widgets.HTML(value='<center><img src="location.png" width="50px"/></center>')
info_image.layout.margin = "0 10px"
location_image.layout.margin = "0 10px"


#background
bg= widgets.HTML(value="<style>.background{width:auto;background-color:grey;}</style>")
output_bg= widgets.HTML(value="<style>.result_bg{width:auto;background-color:grey;text-align:center;}</style>")

vbox_text = widgets.VBox([vbox_result, output_bg])

In [8]:
first_label = widgets.HBox([info_image, tweet_label])
first_label.layout.margin = '0 auto'

second_label = widgets.HBox([location_image, location_label])
second_label.layout.margin = '0 auto'

In [9]:
page = widgets.VBox(children=[logo_image,bg,header_text,first_label,tweet,second_label,location,button_send,vbox_text])
page.add_class("background")
vbox_text.add_class("result_bg")

display(page)

In [10]:
#!pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
